### 批量运行质控任务

In [ ]:
import openpyxl
import boto3
import re

##################################################################
#excel中保存输入文件信息
wb = openpyxl.load_workbook('sample-info.xlsx')
sheet = wb['metagenomic']
#改成你自己的账户ID
account='123456789012'

##################################################################
#任务脚本
script = '/scripts/qc.sh'
#任务信息保存在DynamoDB表中
dbtable = 'metagenomic_qc'
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(dbtable)
#输入文件存储在该S3桶中
bucket = 'metagenomic-' + account + '-cn-northwest-1'

batch = boto3.client('batch')
#Batch任务定义，注意修订版本号，如果有多次部署，可能需要修改
jobdef='jd-qc-test:1'
#Batch任务队列
jobqueue='q-qc-test'

for row in sheet.iter_rows(min_row=2):
    sample = row[0].value
    threads=16
    memory='31500'
    
    # init ddb table
    table.put_item(
       Item={
            'sample': sample,
            'threads': threads,
            'qc_time': 0,
            'status': 0
        }
    )
    
    # submit jobs
    jobname = sample + '-' + str(threads)
    print('jobname: ', jobname, 'queue: ', jobqueue)
    params = {
        'script':script,
        'sample':sample,
        'threads':str(threads),
        'bucket':bucket,
        'dbtable':dbtable
    }
    resourceRequirements = [
        {
            'value':str(threads),
            'type':'VCPU'
        },
        {
            'value':memory,
            'type':'MEMORY'
        },
    ]
    response = batch.submit_job(
        jobName=jobname,
        jobQueue=jobqueue,
        jobDefinition=jobdef,
        parameters=params,
        containerOverrides={
            'resourceRequirements':resourceRequirements
        }
    )
    print(response)